In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [14]:
df = pd.read_csv("C://Users//japje//Documents//AI_project//transformed_ebay_data.csv")

In [15]:
df.head()

,Unnamed: 0,num__Price,num__Feedback Score,cat__Listing Type_Auction,cat__Listing Type_AuctionWithBIN,cat__Listing Type_FixedPrice,cat__Listing Type_StoreInventory,cat__Shipping Type_Calculated,cat__Shipping Type_CalculatedDomesticFlatInternational,cat__Shipping Type_Flat,...,remainder__Large Image URL,remainder__Super Size Image URL,remainder__View Item URL,remainder__Marketplace,remainder__Time duration,remainder__Store URL_flag,remainder__Large Image URL_flag,remainder__Super Size Image URL_flag,remainder__Exchange_rate,remainder__Price_in_USD
0,0,-0.072931,-0.302147,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,https://www.ebay.com/itm/Chicken-Soup-Soul-Chr...,EBAY-US,7.000000,0,0,0,1.0,3.99
1,1,-0.064066,-0.302491,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,https://i.ebayimg.com/00/s/MTUwMFgxMTM4/z/oooA...,https://i.ebayimg.com/00/s/MTUwMFgxMTM4/z/oooA...,https://www.ebay.com/itm/CreativeHavenHome-for...,EBAY-US,61.041667,0,1,1,1.0,6.88
2,2,-0.064802,-0.303912,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,https://i.ebayimg.com/00/s/NTMwWDQ2Nw==/z/BzkA...,https://i.ebayimg.com/00/s/NTMwWDQ2Nw==/z/BzkA...,https://www.ebay.com/itm/Dead-Winter-Beware-Kr...,EBAY-US,29.749769,0,1,1,1.0,6.64
3,3,-0.070170,3.955687,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,https://i.ebayimg.com/00/s/MzE1WDUwMA==/z/S4wA...,https://i.ebayimg.com/00/s/MzE1WDUwMA==/z/S4wA...,https://www.ebay.com/itm/Christmas-Martha-Stew...,EBAY-US,61.041667,1,1,1,1.0,4.89
4,4,-0.029952,-0.297746,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,https://i.ebayimg.com/00/s/MTE4M1gxMDcz/z/qJYA...,https://i.ebayimg.com/00/s/MTE4M1gxMDcz/z/qJYA...,https://www.ebay.com/itm/Lot-10-Christmas-Pict...,EBAY-US,30.000000,0,1,1,1.0,18.00


In [16]:
print(X.columns)


Index(['num__Price', 'num__Feedback Score', 'cat__Listing Type_Auction',
       'cat__Listing Type_AuctionWithBIN', 'cat__Listing Type_FixedPrice',
       'cat__Listing Type_StoreInventory', 'cat__Shipping Type_Calculated',
       'cat__Shipping Type_CalculatedDomesticFlatInternational',
       'cat__Shipping Type_Flat',
       'cat__Shipping Type_FlatDomesticCalculatedInternational',
       'cat__Shipping Type_Free', 'cat__Shipping Type_FreePickup',
       'cat__Shipping Type_Freight', 'cat__Shipping Type_NotSpecified',
       'cat__Top Rated Seller_False', 'cat__Top Rated Seller_True',
       'cat__Condition_Acceptable', 'cat__Condition_Brand New',
       'cat__Condition_Certified - Refurbished',
       'cat__Condition_For parts or not working', 'cat__Condition_Good',
       'cat__Condition_Like New', 'cat__Condition_New',
       'cat__Condition_New other (see details)', 'cat__Condition_New with box',
       'cat__Condition_New with defects', 'cat__Condition_New with tags',
       'c

In [5]:

# Step 2: Prepare the features (X) and the target (y)
# Drop unnecessary columns
X = df.drop(['remainder__Shipping Cost', 'Unnamed: 0'], axis=1)  # Exclude target and unnecessary columns
y = df['remainder__Shipping Cost']  # Target column is 'remainder__Shipping Cost'

# Step 3: Handle missing and infinite values in both features and target

# Convert the features (X) to numeric (force errors to NaN if conversion fails)
X = X.apply(pd.to_numeric, errors='coerce')

# Convert the target variable to numeric (force errors to NaN if conversion fails)
y = pd.to_numeric(y, errors='coerce')

# Check for NaN or infinite values in the target variable (Shipping Cost)
print("Checking for NaN or Infinite values in Shipping Cost:")
print("NaN values in target (y):", y.isna().sum())
print("Infinite values in target (y):", np.isinf(y).sum())

# Check for NaN or infinite values in the features (X)
print("\nChecking for NaN or Infinite values in features (X):")
print("NaN values in features (X):")
print(X.isna().sum())
print("Infinite values in features (X):")
print(np.isinf(X).sum())

# Handle NaN values: You can either drop or impute missing values
# Impute missing values in numerical features (using median) and categorical features (using mode)
X[num_columns] = X[num_columns].fillna(X[num_columns].median())
X[cat_columns] = X[cat_columns].fillna(X[cat_columns].mode().iloc[0])

# Handle NaN values in the target variable (Shipping Cost) - Impute with median
y = y.fillna(y.median())

# Handle Infinite values: Replace them with a large value (e.g., 1e10) for numerical columns
X = X.replace([np.inf, -np.inf], 1e10)
y = y.replace([np.inf, -np.inf], 1e10)

# Step 4: Define the categorical and numerical columns
cat_columns = [col for col in X.columns if col.startswith('cat__')]  # All columns starting with 'cat__'
num_columns = [col for col in X.columns if col.startswith('num__')]  # All columns starting with 'num__'

# Step 5: Define the column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_columns),  # Standardize numerical features
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_columns)  # One-Hot Encoding for categorical features
    ]
)

# Step 6: Create the pipeline with XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)  # Regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 8: Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Step 9: Predict on the test data
y_pred = pipeline.predict(X_test)

# Step 10: Calculate the "accuracy" of predictions with exact matches (100% threshold)
accuracy_threshold = 0  # 100% threshold means exact match
exact_matches = (y_pred == y_test)  # Check if prediction is exactly equal to the true value
accuracy = np.mean(exact_matches)  # Mean of exact matches (as percentage)

# Step 11: Print evaluation results
mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
r2 = r2_score(y_test, y_pred)  # R-squared

print(f"\nModel Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² (R-squared): {r2:.2f}")
print(f"Accuracy (100% threshold): {accuracy * 100:.2f}%")


Checking for NaN or Infinite values in Shipping Cost:
NaN values in target (y): 25977
Infinite values in target (y): 0

Checking for NaN or Infinite values in features (X):
NaN values in features (X):
num__Price                                                     0
num__Feedback Score                                            0
cat__Listing Type_Auction                                      0
cat__Listing Type_AuctionWithBIN                               0
cat__Listing Type_FixedPrice                                   0
cat__Listing Type_StoreInventory                               0
cat__Shipping Type_Calculated                                  0
cat__Shipping Type_CalculatedDomesticFlatInternational         0
cat__Shipping Type_Flat                                        0
cat__Shipping Type_FlatDomesticCalculatedInternational         0
cat__Shipping Type_Free                                        0
cat__Shipping Type_FreePickup                                  0
cat__Shipping Type_

C:\Users\japje\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [27] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [19]:
# Step 10: Calculate "accuracy" of predictions within a tolerance
tolerance = 1.0  # Set a tolerance threshold (e.g., ±1 unit)
exact_matches = np.abs(y_pred - y_test) <= tolerance
accuracy = np.mean(exact_matches)  # Mean of exact matches (as percentage)

# Model evaluation results
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nModel Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² (R-squared): {r2:.2f}")
print(f"Accuracy (within {tolerance} units): {accuracy * 100:.2f}%")



Model Evaluation Metrics:
Mean Absolute Error (MAE): 0.58
Mean Squared Error (MSE): 7.32
R² (R-squared): 0.62
Accuracy (within 1.0 units): 88.10%


In [20]:
#Step 10: Calculate "accuracy" of predictions within a tolerance
tolerance = 1.0  # Set a tolerance threshold (e.g., ±1 unit)
exact_matches = np.abs(y_pred - y_test) <= tolerance
accuracy = np.mean(exact_matches)  # Mean of exact matches (as percentage)

# Model evaluation results
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nModel Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² (R-squared): {r2:.2f}")
print(f"Accuracy (within {tolerance} units): {accuracy * 100:.2f}%")



Model Evaluation Metrics:
Mean Absolute Error (MAE): 0.58
Mean Squared Error (MSE): 7.32
R² (R-squared): 0.62
Accuracy (within 1.0 units): 88.10%


In [12]:
# Step 3: Handle missing and infinite values in both features and target

# Convert the features (X) to numeric (force errors to NaN if conversion fails)
X = X.apply(pd.to_numeric, errors='coerce')

# Convert the target variable to numeric (force errors to NaN if conversion fails)
y = pd.to_numeric(y, errors='coerce')

# Check for NaN or infinite values in the target variable (Shipping Cost)
print("Checking for NaN or Infinite values in Shipping Cost:")
print("NaN values in target (y):", y.isna().sum())
print("Infinite values in target (y):", np.isinf(y).sum())

# Check for NaN or infinite values in the features (X)
print("\nChecking for NaN or Infinite values in features (X):")
print("NaN values in features (X):")
print(X.isna().sum())
print("Infinite values in features (X):")
print(np.isinf(X).sum())

# Handle NaN values: You can either drop or impute missing values
# Impute missing values in numerical features (using median) and categorical features (using mode)
num_columns = [col for col in X.columns if col.startswith('num__')]  # Define numerical columns
cat_columns = [col for col in X.columns if col.startswith('cat__')]  # Define categorical columns

X[num_columns] = X[num_columns].fillna(X[num_columns].median())
X[cat_columns] = X[cat_columns].fillna(X[cat_columns].mode().iloc[0])

# Handle NaN values in the target variable (Shipping Cost) - Impute with median
y = y.fillna(y.median())

# Handle Infinite values: Replace them with a large value (e.g., 1e10) for numerical columns
X = X.replace([np.inf, -np.inf], 1e10)
y = y.replace([np.inf, -np.inf], 1e10)

# Step 4: Handle outliers in numerical columns using the IQR method
def handle_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)  # 25th percentile
        Q3 = df[col].quantile(0.75)  # 75th percentile
        IQR = Q3 - Q1  # Interquartile range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Cap or floor outliers
        df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
    return df

# Apply the outlier handling function to numerical columns
X = handle_outliers(X, num_columns)

# Verify that outliers have been handled
print("\nOutliers handled in numerical features.")


Checking for NaN or Infinite values in Shipping Cost:
NaN values in target (y): 25977
Infinite values in target (y): 0

Checking for NaN or Infinite values in features (X):
NaN values in features (X):
num__Price                                                     0
num__Feedback Score                                            0
cat__Listing Type_Auction                                      0
cat__Listing Type_AuctionWithBIN                               0
cat__Listing Type_FixedPrice                                   0
cat__Listing Type_StoreInventory                               0
cat__Shipping Type_Calculated                                  0
cat__Shipping Type_CalculatedDomesticFlatInternational         0
cat__Shipping Type_Flat                                        0
cat__Shipping Type_FlatDomesticCalculatedInternational         0
cat__Shipping Type_Free                                        0
cat__Shipping Type_FreePickup                                  0
cat__Shipping Type_

In [6]:

# Step 2: Prepare the features (X) and the target (y)
# Drop unnecessary columns
X = df.drop(['remainder__Shipping Cost', 'Unnamed: 0'], axis=1)  # Exclude target and unnecessary columns
y = df['remainder__Shipping Cost']  # Target column is 'remainder__Shipping Cost'

# Step 3: Handle missing and infinite values in both features and target

# Convert the features (X) to numeric (force errors to NaN if conversion fails)
X = X.apply(pd.to_numeric, errors='coerce')

# Convert the target variable to numeric (force errors to NaN if conversion fails)
y = pd.to_numeric(y, errors='coerce')

# Check for NaN or infinite values in the target variable (Shipping Cost)
print("Checking for NaN or Infinite values in Shipping Cost:")
print("NaN values in target (y):", y.isna().sum())
print("Infinite values in target (y):", np.isinf(y).sum())

# Check for NaN or infinite values in the features (X)
print("\nChecking for NaN or Infinite values in features (X):")
print("NaN values in features (X):")
print(X.isna().sum())
print("Infinite values in features (X):")
print(np.isinf(X).sum())

# Handle NaN values: Impute missing values
num_columns = [col for col in X.columns if col.startswith('num__')]  # Define numerical columns
cat_columns = [col for col in X.columns if col.startswith('cat__')]  # Define categorical columns

X[num_columns] = X[num_columns].fillna(X[num_columns].median())  # Impute numerical features with median
X[cat_columns] = X[cat_columns].fillna(X[cat_columns].mode().iloc[0])  # Impute categorical features with mode
y = y.fillna(y.median())  # Impute target variable with median

# Handle Infinite values: Replace with a large finite value
X = X.replace([np.inf, -np.inf], 1e10)
y = y.replace([np.inf, -np.inf], 1e10)

# Step 4: Handle outliers in numerical columns using the IQR method
def handle_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)  # 25th percentile
        Q3 = df[col].quantile(0.75)  # 75th percentile
        IQR = Q3 - Q1  # Interquartile range
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        # Cap or floor outliers
        df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
    return df

# Apply the outlier handling function to numerical columns
X = handle_outliers(X, num_columns)

# Step 5: Define the column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_columns),  # Standardize numerical features
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_columns)  # One-Hot Encoding for categorical features
    ]
)

# Step 6: Create the pipeline with XGBoost model
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)  # Regression model
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 8: Fit the pipeline to the training data
pipeline.fit(X_train, y_train)

# Step 9: Predict on the test data
y_pred = pipeline.predict(X_test)

# Step 10: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)  # Mean Absolute Error
mse = mean_squared_error(y_test, y_pred)  # Mean Squared Error
r2 = r2_score(y_test, y_pred)  # R-squared

print(f"\nModel Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² (R-squared): {r2:.2f}")


Checking for NaN or Infinite values in Shipping Cost:
NaN values in target (y): 25977
Infinite values in target (y): 0

Checking for NaN or Infinite values in features (X):
NaN values in features (X):
num__Price                                                     0
num__Feedback Score                                            0
cat__Listing Type_Auction                                      0
cat__Listing Type_AuctionWithBIN                               0
cat__Listing Type_FixedPrice                                   0
cat__Listing Type_StoreInventory                               0
cat__Shipping Type_Calculated                                  0
cat__Shipping Type_CalculatedDomesticFlatInternational         0
cat__Shipping Type_Flat                                        0
cat__Shipping Type_FlatDomesticCalculatedInternational         0
cat__Shipping Type_Free                                        0
cat__Shipping Type_FreePickup                                  0
cat__Shipping Type_

C:\Users\japje\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [27] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [7]:

# Step 11: Calculate accuracy based on 10% tolerance
tolerance = 0.1  # 10%
within_tolerance = np.abs(y_pred - y_test) <= (tolerance * y_test)
accuracy = np.mean(within_tolerance) * 100  # Convert to percentage

# Print evaluation metrics
print(f"\nModel Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² (R-squared): {r2:.2f}")
print(f"Accuracy (within ±10% of true value): {accuracy:.2f}%")



Model Evaluation Metrics:
Mean Absolute Error (MAE): 0.60
Mean Squared Error (MSE): 7.69
R² (R-squared): 0.60
Accuracy (within ±10% of true value): 6.83%


In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb

# Step 1: Load the dataset
df = pd.read_csv("C://Users//japje//Documents//AI_project//transformed_ebay_data.csv")

# Step 2: Prepare the features (X) and the target (y)
# Drop unnecessary columns
X = df.drop(['remainder__Shipping Cost', 'Unnamed: 0'], axis=1)  # Exclude target and unnecessary columns
y = df['remainder__Shipping Cost']  # Target column is 'remainder__Shipping Cost'

# Step 3: Handle missing and infinite values
X = X.apply(pd.to_numeric, errors='coerce')
y = pd.to_numeric(y, errors='coerce')

num_columns = [col for col in X.columns if col.startswith('num__')]
cat_columns = [col for col in X.columns if col.startswith('cat__')]

X[num_columns] = X[num_columns].fillna(X[num_columns].median())
X[cat_columns] = X[cat_columns].fillna(X[cat_columns].mode().iloc[0])
y = y.fillna(y.median())

X = X.replace([np.inf, -np.inf], 1e10)
y = y.replace([np.inf, -np.inf], 1e10)

# Step 4: Handle outliers in numerical columns using the IQR method
def handle_outliers(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df[col] = df[col].clip(lower=lower_bound, upper=upper_bound)
    return df

X = handle_outliers(X, num_columns)

# Step 5: Define the column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_columns),
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_columns)
    ]
)

# Step 6: Create the pipeline
model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Step 7: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 8: Define hyperparameter grid
param_grid = {
    'model__learning_rate': [0.01, 0.05, 0.1],
    'model__max_depth': [3, 5, 7],
    'model__n_estimators': [100, 300, 500],
    'model__min_child_weight': [1, 3, 5],
    'model__colsample_bytree': [0.5, 0.7, 1.0]
}

# Step 9: Perform GridSearchCV
grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=3,  # 3-fold cross-validation
    scoring='neg_mean_squared_error',  # Evaluate based on MSE
    verbose=1,
    n_jobs=-1  # Use all processors
)

# Fit GridSearch to training data
grid_search.fit(X_train, y_train)

# Best parameters and score
print("\nBest Parameters from GridSearchCV:")
print(grid_search.best_params_)

# Step 10: Predict using the best model
best_pipeline = grid_search.best_estimator_
y_pred = best_pipeline.predict(X_test)

# Step 11: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Accuracy within ±10%
tolerance = 0.1
within_tolerance = np.abs(y_pred - y_test) <= (tolerance * y_test)
accuracy = np.mean(within_tolerance) * 100

print(f"\nModel Evaluation Metrics:")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"R² (R-squared): {r2:.2f}")
print(f"Accuracy (within ±10% of true value): {accuracy:.2f}%")


Fitting 3 folds for each of 243 candidates, totalling 729 fits

Best Parameters from GridSearchCV:
{'model__colsample_bytree': 0.7, 'model__learning_rate': 0.1, 'model__max_depth': 7, 'model__min_child_weight': 5, 'model__n_estimators': 300}


C:\Users\japje\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [27] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(



Model Evaluation Metrics:
Mean Absolute Error (MAE): 0.60
Mean Squared Error (MSE): 7.52
R² (R-squared): 0.61
Accuracy (within ±10% of true value): 6.91%
